In [2]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from config import interim_data_path, masks_path

In [3]:
# sensors = [f'R{i}' for i in range(1,13)]
# datasets = ['trn', 'vld', 'tst']

# for sensor in sensors:
#     X_trn = pd.read_feather(interim_data_path / f'{sensor}-trn.feather')
#     X_vld = pd.read_feather(interim_data_path / f'{sensor}-vld.feather')
#     break

In [4]:
targets = [
    'NO_conc',
    'CH4_conc',
    'H2S_conc',
    'SO2_conc',
    'HCOH_conc',
    'CO_conc',
    'H2_conc',
    'NO2_conc',
    'NH3_conc'
    ]

In [5]:
df_trn = pd.read_feather(interim_data_path / 'trn.feather')
df_vld = pd.read_feather(interim_data_path / 'vld.feather')
df_tst = pd.read_feather(interim_data_path / 'tst.feather')

In [6]:
scaler = MinMaxScaler()
scaler.fit(df_trn)
df_trn = pd.DataFrame(scaler.transform(df_trn), columns=df_trn.columns)
df_vld = pd.DataFrame(scaler.transform(df_vld), columns=df_vld.columns)
df_tst = pd.DataFrame(scaler.transform(df_tst), columns=df_tst.columns)

In [7]:
X_trn = df_trn.drop(columns=targets)
y_trn = df_trn[targets]
X_vld = df_vld.drop(columns=targets)
y_vld = df_vld[targets]
X_tst = df_tst.drop(columns=targets)
y_tst = df_tst[targets]

In [8]:
# pd.read_feather(masks_path / 'R1.feather')

In [9]:
from sklearn.linear_model import Ridge, Lasso


In [10]:
for target in targets:
    model = Lasso(alpha=1.0)
    model.fit(X_trn, y_trn[target])
    print(model.score(X_vld, y_vld[target]))

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [15]:
X_trn.describe()

,R1_0,R1_1,R1_2,R1_3,R1_4,R1_5,R1_6,R1_7,R1_8,R1_9,...,R12_1191,R12_1192,R12_1193,R12_1194,R12_1195,R12_1196,R12_1197,R12_1198,R12_1199,R12_1200
count,1890.000000,1890.000000,1890.000000,1890.000000,1890.000000,1890.000000,1890.000000,1890.000000,1890.000000,1890.000000,...,1890.000000,1890.000000,1890.000000,1890.000000,1890.000000,1890.000000,1890.000000,1890.000000,1890.000000,1890.000000
mean,0.117529,0.117195,0.116892,0.117117,0.116067,0.115989,0.114937,0.115383,0.114587,0.114862,...,0.385393,0.384879,0.384369,0.383899,0.383497,0.383054,0.382668,0.382184,0.381808,0.381527
std,0.189606,0.189043,0.188613,0.188978,0.187376,0.187280,0.185611,0.186331,0.185067,0.185500,...,0.356786,0.356473,0.356139,0.355792,0.355514,0.355223,0.354908,0.354665,0.354427,0.354168
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.008797,0.008774,0.008822,0.008860,0.008824,0.008826,0.008813,0.008890,0.008882,0.008938,...,0.078874,0.078908,0.078897,0.078908,0.079000,0.078959,0.080068,0.078994,0.079057,0.078984
50%,0.044765,0.044653,0.044561,0.044788,0.044349,0.044399,0.044072,0.044532,0.044106,0.044378,...,0.252818,0.253026,0.252670,0.252656,0.252153,0.252211,0.251955,0.251937,0.251890,0.251566
75%,0.113379,0.112990,0.112616,0.112776,0.111970,0.111698,0.110780,0.111221,0.110380,0.110864,...,0.675784,0.673965,0.673480,0.670679,0.669172,0.666577,0.664016,0.662877,0.660351,0.658579
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [16]:
X_vld.describe()

,R1_0,R1_1,R1_2,R1_3,R1_4,R1_5,R1_6,R1_7,R1_8,R1_9,...,R12_1191,R12_1192,R12_1193,R12_1194,R12_1195,R12_1196,R12_1197,R12_1198,R12_1199,R12_1200
count,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,6.300000e+02,630.000000,630.000000,630.000000,...,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000
mean,0.118043,0.117696,0.117399,0.117619,0.116539,0.116455,1.153978e-01,0.115798,0.115026,0.115270,...,0.401678,0.400777,0.400219,0.399744,0.399154,0.398771,0.398246,0.398040,0.397582,0.397062
std,0.183154,0.182556,0.182170,0.182525,0.180868,0.180842,1.791940e-01,0.179745,0.178596,0.178914,...,0.362266,0.361872,0.361654,0.361333,0.361171,0.360794,0.360684,0.360232,0.359979,0.359887
min,0.000002,0.000003,0.000002,0.000001,0.000002,0.000001,-6.484438e-07,-0.000001,-0.000001,-0.000002,...,0.001208,0.001178,0.001161,0.001131,0.001167,0.001185,0.001203,0.001214,0.001209,0.001130
25%,0.009091,0.009063,0.009060,0.009117,0.009052,0.009088,9.018217e-03,0.009116,0.009061,0.009150,...,0.079212,0.079393,0.079208,0.079837,0.079356,0.079220,0.079140,0.079142,0.079240,0.079231
50%,0.053037,0.052892,0.052695,0.052981,0.052621,0.052681,5.236070e-02,0.052672,0.052574,0.052653,...,0.280171,0.280100,0.276667,0.276684,0.276443,0.276254,0.276118,0.279050,0.278484,0.278169
75%,0.114759,0.114494,0.114094,0.114704,0.113117,0.113386,1.124015e-01,0.112825,0.112095,0.111814,...,0.744844,0.740999,0.738726,0.736033,0.734376,0.729837,0.728445,0.727684,0.725482,0.722867
max,0.942950,0.939772,0.939702,0.941301,0.933209,0.936443,9.334217e-01,0.934652,0.930642,0.937625,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [17]:
X_tst.describe()

,R1_0,R1_1,R1_2,R1_3,R1_4,R1_5,R1_6,R1_7,R1_8,R1_9,...,R12_1191,R12_1192,R12_1193,R12_1194,R12_1195,R12_1196,R12_1197,R12_1198,R12_1199,R12_1200
count,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,...,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000
mean,0.117816,0.117475,0.117150,0.117385,0.116292,0.116201,0.115113,0.115566,0.114725,0.115034,...,0.394644,0.394093,0.393493,0.393017,0.392473,0.392047,0.391589,0.391172,0.390678,0.390387
std,0.181393,0.180842,0.180339,0.180801,0.179107,0.179027,0.177299,0.178107,0.176710,0.177233,...,0.358048,0.357772,0.357422,0.357093,0.356807,0.356485,0.356144,0.355894,0.355591,0.355385
min,-0.000005,-0.000003,-0.000004,-0.000006,-0.000005,-0.000005,-0.000007,-0.000007,-0.000008,-0.000008,...,0.000859,0.000863,0.000863,0.000862,0.000860,0.000859,0.000861,0.000861,0.000857,0.000857
25%,0.009082,0.009059,0.009059,0.009120,0.009054,0.009082,0.009027,0.009099,0.009079,0.009140,...,0.079127,0.079073,0.079110,0.079089,0.079051,0.079085,0.078998,0.079125,0.079043,0.079022
50%,0.054633,0.054497,0.054484,0.054719,0.054289,0.054382,0.053935,0.054105,0.053996,0.054227,...,0.279477,0.279761,0.279093,0.279184,0.278129,0.277987,0.277868,0.277656,0.276646,0.276884
75%,0.114611,0.114153,0.114003,0.113830,0.113026,0.112452,0.111788,0.111942,0.111442,0.111998,...,0.708980,0.706963,0.705002,0.701568,0.699962,0.697356,0.694632,0.695461,0.689950,0.689532
max,0.943728,0.944774,0.941120,0.953304,0.938214,0.945322,0.934074,0.945212,0.936552,0.940183,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [19]:
for target in targets:
    model = Ridge(alpha=1.0)
    model.fit(X_trn, y_trn[target])
    print(f'{target} : {model.score(X_vld, y_vld[target]):0.3f}')

NO_conc : 0.990
CH4_conc : 0.896
H2S_conc : 0.977
SO2_conc : 0.679
HCOH_conc : 0.943
CO_conc : 0.931
H2_conc : 0.925
NO2_conc : 0.989
NH3_conc : 1.000
